In [46]:
""" airportデータ作成 """

# route.csvを読み込む
route_data = []
with open('route_airplane.csv', 'r') as f:
    for line in f:
        l = line.strip().replace(' - ', ', ')
        l = l.strip().split(', ')

        if len(l) < 2:
            continue

        route_data.append(l)

# airport_code.csvを読み込む
airport_code_data = []
with open('airport_code.csv', 'r') as f:
    for line in f:
        airport_code_data.append(line.strip().split(', '))

# route_dattaに空港コードがなければ追加
output = []
COMPANY = 'SFJ'
COLUMN = 5
for line in route_data:
    if len(line) == COLUMN:
        output.append(line)
        continue

    if len(line) == 2:
        l = [COMPANY] + [l_ for l_ in line]
        l.append('')
        l.append('')

        # airport_code.csvから空港コードを取得
        for airport_code_line in airport_code_data:
            if line[0] in airport_code_line[3]:
                l[3] = airport_code_line[0]
            if line[1] in airport_code_line[3]:
                l[4] = airport_code_line[0]
        
        output.append(l)

# route.csvとして保存
with open('route_airplane.csv', 'w') as f:
    for line in output:
        f.write(', '.join(line) + '\n')

In [131]:
""" コスト計算 """

import json
import networkx as nx

# Load route json
# airport
with open('route_airport.json', 'r') as f:
    airport_data = json.load(f)

# shinkansen
with open('route_shinkansen.json', 'r') as f:
    shinkansen_data = json.load(f)

# airport_shinkansen
with open('route_airport_shinkansen.json', 'r') as f:
    airport_shinkansen_data = json.load(f)


# Create Graph 
G = nx.Graph()

# airport
for key_company in airport_data.keys():
    for key_from in airport_data[key_company].keys():
        for key_to in airport_data[key_company][key_from].keys():
            G.add_edge(key_from, key_to, weight=airport_data[key_company][key_from][key_to]['cost'])

# shinkansen
for key_company in shinkansen_data.keys():
    for key_from in shinkansen_data[key_company].keys():
        for key_to in shinkansen_data[key_company][key_from].keys():
            G.add_edge(key_from, key_to, weight=shinkansen_data[key_company][key_from][key_to]['cost'])

# airport_shinkansen
for from_ in airport_shinkansen_data.keys():
    for to_ in airport_shinkansen_data[from_].keys():
        G.add_edge(from_, to_, weight=airport_shinkansen_data[from_][to_]['cost'])


# FUKからHKDまでの経路をコストが少ない順に5つ求める
start_node = ['FUK', '博多']
goal_node = ['AOJ', 'MSJ', 'HKD', '七戸']
num = 5
routes = []
for st in start_node:
    for gl in goal_node:
        print(f"---- {st} -> {gl} ----")

        # 最短経路を5つ求める
        for i, path in enumerate(nx.shortest_simple_paths(G, st, gl, weight='weight')):
            if i >= num:
                break

            # コストを計算
            cost = 0
            l = []
            for j in range(len(path)-1):
                c = G[path[j]][path[j+1]]['weight']
                cost += c 

                l.append([path[j+1], c])


            routes.append([path, cost])
            print(path, cost)

# コストが少ない順にソート
routes.sort(key=lambda x:x[1])

print('---- Best Route ----')
for route in routes[:10]:
    print(route)


---- FUK -> AOJ ----
['FUK', 'ITM', 'AOJ'] 29580
['FUK', 'NRT', '東京', '八戸', 'AOJ'] 33110
['FUK', 'NRT', '東京', '七戸', 'AOJ'] 33330
['FUK', 'NRT', '東京', '上野', '八戸', 'AOJ'] 33920
['FUK', 'NRT', '東京', '上野', '七戸', 'AOJ'] 34140
---- FUK -> MSJ ----
['FUK', 'NRT', '東京', '八戸', 'MSJ'] 33110
['FUK', 'NRT', '東京', '七戸', 'MSJ'] 33330
['FUK', 'NRT', '東京', 'HND', 'MSJ'] 33375
['FUK', 'NRT', '東京', '上野', '八戸', 'MSJ'] 33920
['FUK', 'NRT', '東京', '上野', '七戸', 'MSJ'] 34140
---- FUK -> HKD ----
['FUK', 'NRT', '東京', 'HND', 'HKD'] 37330
['FUK', 'NGO', 'HKD'] 37670
['FUK', 'NRT', '東京', '上野', 'HND', 'HKD'] 38350
['FUK', 'NRT', '東京', '品川', 'HND', 'HKD'] 38360
['FUK', 'HND', 'HKD'] 39280
---- FUK -> 七戸 ----
['FUK', 'NRT', '東京', '七戸'] 30330
['FUK', 'NRT', '東京', '上野', '七戸'] 31140
['FUK', 'NRT', '東京', '大宮', '七戸'] 31520
['FUK', 'NRT', '東京', '八戸', '七戸'] 31640
['FUK', 'NRT', '東京', '上野', '大宮', '七戸'] 32250
---- 博多 -> AOJ ----
['博多', 'FUK', 'ITM', 'AOJ'] 32580
['博多', '小倉', 'FUK', 'ITM', 'AOJ'] 34690
['博多', 'FUK', 'NRT', '東京

In [132]:
""" shinkansenデータ作成 """

import json

# Load route_shinkansen.csv
route_data = []
with open('route_shinkansen.csv', 'r') as f:
    for line in f:
        route_data.append(line.strip().split(', '))

# Load route_shinkansen.json
with open('route_shinkansen.json', 'r') as f:
    json_ = json.load(f)

COMPANY = "東北新幹線"
json_[COMPANY] = {}
for line in route_data[0][1:]:
    json_[COMPANY][line] = {}

for line in route_data[1:]:
    sta = line[0]

    for i in range(1, len(line)):
        sta2 = route_data[0][i]

        if sta == sta2:
            continue
        print(sta, sta2, line[i])

        json_[COMPANY][sta][sta2] = {
            "cost": int(line[i]),
            "time": 0
        }

# Write json 
with open('route_shinkansen.json', 'w') as f:
    json.dump(json_, f, ensure_ascii=False, indent=4)


東京 上野 1020
東京 大宮 1620
東京 小山 4060
東京 宇都宮 4930
東京 那須塩原 5910
東京 新白河 6670
東京 郡山 8200
東京 福島 8950
東京 白石蔵王 10350
東京 仙台 10890
東京 古川 11430
東京 くりこま 12280
東京 一ノ関 12820
東京 水沢江刺 13040
東京 北上 13360
東京 新花巻 13360
東京 盛岡 14230
東京 いわて 14770
東京 二戸 15420
東京 八戸 15780
東京 七戸 16000
東京 新青森 16840
上野 東京 1020
上野 大宮 1330
上野 小山 3680
上野 宇都宮 4720
上野 那須塩原 5700
上野 新白河 6460
上野 郡山 7990
上野 福島 8740
上野 白石蔵王 10140
上野 仙台 10680
上野 古川 11220
上野 くりこま 12070
上野 一ノ関 12610
上野 水沢江刺 12830
上野 北上 13150
上野 新花巻 13150
上野 盛岡 14020
上野 いわて 14560
上野 二戸 15210
上野 八戸 15570
上野 七戸 15790
上野 新青森 16630
大宮 東京 1620
大宮 上野 1330
大宮 小山 1950
大宮 宇都宮 3680
大宮 那須塩原 5380
大宮 新白河 5700
大宮 郡山 6460
大宮 福島 8420
大宮 白石蔵王 8740
大宮 仙台 10360
大宮 古川 11000
大宮 くりこま 11220
大宮 一ノ関 12070
大宮 水沢江刺 12290
大宮 北上 12610
大宮 新花巻 12830
大宮 盛岡 13480
大宮 いわて 14020
大宮 二戸 14560
大宮 八戸 15570
大宮 七戸 15570
大宮 新青森 16110
小山 東京 4060
小山 上野 3680
小山 大宮 1950
小山 宇都宮 1360
小山 那須塩原 3640
小山 新白河 5050
小山 郡山 5700
小山 福島 6460
小山 白石蔵王 7990
小山 仙台 8740
小山 古川 10140
小山 くりこま 10360
小山 一ノ関 11000
小山 水沢江刺 11220
小山 北上 12070
小山 新花巻 120

In [133]:
""" 都道府県名データ作成 """

import csv
import json
import os
import requests

def get_prefecture_from_name(name: str, api_key: str):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": name,
        "key": api_key,
        "language": "ja"
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if "results" in data and len(data["results"]) > 0:
        address_components = data["results"][0]["address_components"]
        for component in address_components:
            if "administrative_area_level_1" in component["types"]:
                return component["long_name"]

    return None

# .env.localからAPIキーを取得
api_key = os.environ['GOOGLE_API_KEY']

# Load route_airport
with open('airport_code.csv', 'r') as f:
    reader = csv.reader(f)
    airport_data = [row for row in reader]

# Load route_shinkansen.json
with open('route_shinkansen.json', 'r') as f:
    shinkansen_data = json.load(f)

route_data = {}

for line in airport_data:
    prefecture = get_prefecture_from_name(line[2], api_key)
    route_data[line[0]] = prefecture

for key_company in shinkansen_data.keys():
    for key_from in shinkansen_data[key_company].keys():
        prefecture = get_prefecture_from_name(key_from, api_key)
        route_data[key_from] = prefecture 

# Write landmarks.json
with open('landmarks.json', 'w') as f:
    json.dump(route_data, f, ensure_ascii=False, indent=4)


In [134]:
""" airportとshinkansenのデータを結合 """

# Load route_airport.json
with open('route_airport.json', 'r') as f:
    airport_data = json.load(f)

# Load route_shinkansen.json
with open('route_shinkansen.json', 'r') as f:
    shinkansen_data = json.load(f)

# Load landmarks.json
with open('landmarks.json', 'r') as f:
    landmarks_data = json.load(f)

route_data = {}
for key_company in airport_data.keys():
    for key_from in airport_data[key_company].keys():
        for key_company_2 in shinkansen_data.keys():
            for key_from_2 in shinkansen_data[key_company_2].keys():
                if landmarks_data[key_from] == landmarks_data[key_from_2]:
                    if key_from in route_data:
                        route_data[key_from][key_from_2] = {"cost": 3000, "time": 0}
                    else:
                        route_data[key_from] = {key_from_2: {"cost": 3000, "time": 0}}

route_data["東京"] = {"NRT": {"cost": 3000, "time": 0}}
route_data["NRT"] = {"東京": {"cost": 3000, "time": 0}}

# Write route_data.json
with open('route_airport_shinkansen.json', 'w') as f:
    json.dump(route_data, f, ensure_ascii=False, indent=4)



In [135]:
""" airportのcostを調整 """

from_ = 'ITM'
to_ = 'FUK'
company = ''
cost = 10000 

# route_airport.jsonを読み込む
with open('route_airport.json', 'r') as f:
    airport_data = json.load(f)

if company == '':
    # 全ての会社のcostを変更
    for company in airport_data.keys():
        if from_ in airport_data[company]:
            if to_ in airport_data[company][from_]:
                airport_data[company][from_][to_]['cost'] = cost
        
        if to_ in airport_data[company]:
            if from_ in airport_data[company][to_]:
                airport_data[company][to_][from_]['cost'] = cost
else:
    airport_data[company][from_][to_]['cost'] = cost
    airport_data[company][to_][from_]['cost'] = cost

# route_data.jsonを保存
# 文字化けしないように
with open('route_airport.json', 'w') as f:
    json.dump(airport_data, f, ensure_ascii=False, indent=4)

